In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [3]:
dataObject = DataProcess()

# static measurement jacobian
static_ind = [0,1,2,4,5,6,8,9,10]
static_Nt = 22
static_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(static_Nt)+'_scale.csv')
num_static = len(static_ind)

static_Q = []
for i in static_ind:
    static_Q.append(dataObject.split_jaco(static_jaco, i,static_Nt))

    
    
dynamic_ind = [3,7,11,12,13]
dynamic_Nt = 5
num_dynamic = len(dynamic_ind)
dynamic_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(dynamic_Nt)+'_scale.csv')

dynamic_Q = []
for i in dynamic_ind:
    dynamic_Q.append(dataObject.split_jaco(dynamic_jaco, i, dynamic_Nt))
    
print("static Q shape:", np.shape(static_Q))
print("dynamic Q shape:", np.shape(dynamic_Q))

jacobian shape: (308, 5)
jacobian shape: (70, 5)
static Q shape: (9, 22, 5)
dynamic Q shape: (5, 5, 5)


In [4]:
Q = []
for i in range(len(static_ind)):
    Q.append(static_Q[i])
for j in range(len(dynamic_ind)):
    Q.append(dynamic_Q[j])

In [5]:
print(np.shape(Q[1]))
print(Q[13])

(22, 5)
[[-0.2369589708548623, -0.0160502930881647, 0.0146807209797872, -0.1763506246058283, 0.2949746088336674], [-0.2369521215057689, -0.0160502930881647, 0.0146807209797872, -0.1763506246058283, 0.2949746088336674], [-0.0695822811896183, -0.0134711387931862, 0.0603797016684929, -0.053746313047065, 0.0260785419601256], [0.1039139858946325, -0.0103535467732923, 0.1023829539095281, 0.0693874136743377, -0.2447841359458552], [0.1671865305544889, -0.0092458200384041, 0.1147703544223888, 0.1104832525762961, -0.3385565785691546]]


In [6]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]
static_idx_upd = [0,1,2,3,4,5,6,7,8]
dynamic_idx_upd = [9,10,11,12,13]

num_param = 5

num_total = num_static + num_dynamic*dynamic_Nt

In [7]:
calculator = MeasurementOptimizer(Q, static_idx_upd, dynamic_idx_upd, num_param, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(5,)
9 0
(22, 1)


ValueError: cannot reshape array of size 110 into shape (1,5)

In [8]:
print(calculator.Sigma_inv[(9,0)])
print(calculator.Q_dynamic_flatten[0])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.539636670963493, -0.0022541710490597, -0.0498629482614828, 1.9351604614661069, 0.5513282116769436], [-0.3340760407153356, 0.1021652968364613, 0.1016944838159672, 2.3403863441379817, 0.221939518269873], [-0.3494661138571784, 0.1120662029219454, 0.2550896605274281, 2.0027607969742065, 0.1809770476947275], [-0.4678795656821666, 0.0789891670526334, 0.3012354877648384, 1.6852339585553322, 0.3702798007923669], [-0.3487072812830348, 0.1544493767900593, 0.3679533495956177, 2.016853552909431, 0.1889221494208309], [-0.2149867536290262, 0.1729367514893193, 0.4172631902483755, 2.2827109973126425, -0.0128215863177127], [-0.0597343516865485, 0.1787397054563655, 0.4592339471516512, 2.0431098799382568, -0.2109020116805193], [-0.2295705671608305, 0.0878631069081999, 0.393229772557104, 0.9576582753596296, 0.0872533926070839], [-0.1234288608689441, 0.0230056295989378, 

In [9]:
print(calculator.dynamic_to_flatten[1])

[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


In [ ]:
print(np.shape(calculator.Q[0]))
print(np.shape(np.asarray(calculator.Q[9]).reshape(5,1)))
print(np.shape(calculator.Sigma_inv[(0,9)]))

print(calculator.total_no_measure)

### Set up grey-box 

In [ ]:
from greybox_generalize import LogDetModel

In [ ]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [ ]:
cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         200, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         200, #des.gas_outlet.T (6)
         600, #ads.T19 (8)
         600, #ads.T23 (9)
         600 #ads.T28 (10)
       ]

for i in range(dynamic_Nt*num_dynamic):
    cost.append(10)


In [ ]:
#print(np.shape(ans_y))

init_y = np.random.rand(num_total)

update_y = [] 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

In [ ]:
mip_option = False
objective = "D"
budget_opt = 5000
sparse_opt = False
fix_opt = False

In [ ]:
mod = calculator.continuous_optimization(cost, mixed_integer=mip_option, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt,
                                        init_fim = fim_prior)

In [ ]:

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)

elif mip_option and objective=="A":
    solver = pyo.SolverFactory('mindtpy')
    results = solver.solve(mod, tee=True, integer_tolerance=0.5)

elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)

In [ ]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

In [ ]:
# problem size calculator
if True:
    nr = num_total
    npa = 5
    num_var = (nr*nr+nr)/2 + npa*npa + 1
    num_equal = npa*npa+1
    num_ineq = 1.5*(nr**2-nr)+1

    print(num_var, num_equal, num_ineq)

In [ ]:
import numpy as np

ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])

for i in range(num_total):
    for j in range(num_total):
        pyomo_y[i,j] = pyo.value(mod.cov_y[i,j])
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))

In [ ]:
figure = plt.figure()
axes = figure.add_subplot(111)

caxes = axes.matshow(ans_y, interpolation='nearest')
figure.colorbar(caxes)

plt.show()

print(np.shape(ans_y))

In [ ]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

In [ ]:
import pickle

store = False

if store:
    file = open('Fe23_D_110_3000', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [ ]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    print(dynamic_name[r], ": ", sol_y[r])

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
#measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
#measure_name = ['Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
measure_name = ['a', 'b', 'c', 'd', 'e']

#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(20, 2200, number_t)
time_set = [str(i) for i in time_set]
print(len(time_set), len(measure_name))
print(np.shape(sol_y))

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, time_set, measure_name, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [65693.94, 66574.63, 67474.485, 67608.817]
d_set = [np.exp(30.13), np.exp(30.719), np.exp(31.1205), np.exp(31.2586)]

plot_data([3000,4000,5000,6400], a_set, d_set)